### Week 2 Homework

For questions 1 and 2: 

Do a regression to estimate the fixed effect of each group. We assume that there is one single linear coefficient for all the data, plus the fixed effect of each group. Use the file homework_2.1.csv.  The variables G1, G2, and G3 are the outcomes and the time is the treatment.

In [14]:
import pandas as pd
import numpy as np
import os
import sys

In [43]:
# read the homework 2.1 file
df_1 = pd.read_csv(r'C:\Users\JT von Seggern\DS Masters Repos\2025-summer-mod-6\homework_2.1.csv')
df_1 = df_1.drop(columns=['Unnamed: 0'])

In [44]:
df_1.head()

,time,G1,G2,G3
0,0,0.882026,1.441575,0.065409
1,1,0.210079,-0.163880,0.140310
2,2,0.509369,-0.115242,0.819830
3,3,1.150447,1.014698,0.607632
4,4,0.973779,-0.046562,0.610066


In [45]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    100 non-null    int64  
 1   G1      100 non-null    float64
 2   G2      100 non-null    float64
 3   G3      100 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 3.3 KB


In [46]:
df_1.describe()

,time,G1,G2,G3
count,100.000000,100.000000,100.000000,100.000000
mean,49.500000,0.524904,1.036006,0.715384
std,29.011492,0.561614,0.552593,0.581227
min,0.000000,-1.076495,-0.163880,-0.508478
25%,24.750000,0.186046,0.621088,0.298386
50%,49.500000,0.464842,1.006778,0.643041
75%,74.250000,0.947191,1.427074,1.125461
max,99.000000,1.862935,2.561618,2.321958


In [ ]:
# plot the data
import matplotlib.pyplot as plt
import seaborn as sns